In [1]:
from collections import namedtuple
import os
os.chdir("..")
os.getcwd()


'd:\\ineuron\\DL_project\\CNN_project'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbackPreparationConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
  

In [3]:
from cnnProject.constant import *
from cnnProject.utils import read_yaml,create_directories

In [4]:
class ConfigrationManger:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifact_root])

    def get_callback_perparation_config(self) -> CallbackPreparationConfig :
        config = self.config.callback_preparation

        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_checkpoint_dir)
        ])

        callback_prepare_config = CallbackPreparationConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return callback_prepare_config


In [5]:
#CallbackPrepareComponent
import os
import time
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class CallbackPreparation:
    def __init__(self,config:CallbackPreparationConfig) -> None:
        self.config = config

    @property
    def _create_tb_callbacks(self):
        #good ideal to create a folder structure with timestamps
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")

        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True  #save only best weights aka loss was less
        )



    def get_tb_checkpoint_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

   
    

In [7]:
try:
    config = ConfigrationManger()
    callback_preparation_config = config.get_callback_perparation_config()
    callback_prepare = CallbackPreparation(config=callback_preparation_config)
    callback_lst = callback_prepare.get_tb_checkpoint_callback()
except Exception as e:
    raise e